In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import shutil
from tqdm import tqdm
import math
import cv2
# import mediapipe as mp

In [5]:
# Settings
VIDEOS_DIR    = '/kaggle/working/SL_retained_classes'
OUTPUT_IMG_DIR = "/kaggle/working/SL_frames_output"
FRAMES_PER_SECOND = 5                    
MAX_FRAMES_PER_VIDEO  = 37
TRIM_START_PORTION = 0.15
TRIM_END_PORTION = 0.85
BATCH_SIZE  = 32
LR_HEAD     = 1e-4
LR_FINE     = 1e-5
EPOCH_HEAD1 = 2    # quick head-only to gauge per-class F1
EPOCH_HEAD2 = 3    # head-only for final training
EPOCH_FINE  = 7    # fine-tune epochs
VAL_SIZE    = 0.2
DEVICE      = 'cuda' if torch.cuda.is_available() else 'cpu'

NameError: name 'torch' is not defined

In [ ]:
df_retained_classes = pd.read_csv("/kaggle/input/retained-classes/one_hand_classes_list_re_classified.csv")

In [27]:
# where the dataset is located
SOURCE_DIR = "/kaggle/input/sign-language-dataset-wlasl-videos/dataset/SL"
TARGET_DIR = "/kaggle/working/SL_retained_classes"

os.makedirs(TARGET_DIR, exist_ok=True)

classes_to_keep = set(df_retained_classes['class_name'].unique())

for folder in tqdm(os.listdir(SOURCE_DIR)):
    src_path = os.path.join(SOURCE_DIR, folder)

    if folder in classes_to_keep:
        dst_path = os.path.join(TARGET_DIR, folder)
        shutil.copytree(src_path, dst_path)   # copy entire folder

100%|██████████| 2000/2000 [00:10<00:00, 182.91it/s]


In [ ]:
os.makedirs(OUTPUT_IMG_DIR, exist_ok=True)

class_list = [c for c in os.listdir(VIDEOS_DIR) if os.path.isdir(os.path.join(VIDEOS_DIR, c))]

bad_videos = []

for cls in tqdm(class_list, desc="Processing SL Classes", ncols=100):
    class_path = os.path.join(VIDEOS_DIR, cls)
    out_class_dir = os.path.join(OUTPUT_IMG_DIR, cls)
    os.makedirs(out_class_dir, exist_ok=True)

    videos = [v for v in os.listdir(class_path) if v.lower().endswith((".mp4", ".avi", ".mov", ".mkv"))]

    for video in videos:
        video_path = os.path.join(class_path, video)

        cap = cv2.VideoCapture(video_path)

        if not cap.isOpened():
            bad_videos.append(video_path)
            continue

        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        if fps <= 1 or total_frames <= 1:
            bad_videos.append(video_path)
            cap.release()
            continue

        frame_interval = max(int(fps / FRAMES_PER_SECOND), 1)

        frames = []
        frame_num = 0

        while True:
            ret, frame = cap.read()

            if not ret:
                break

            if frame_num % frame_interval == 0:
                frames.append(frame)

            frame_num += 1

        cap.release()

        # Skip empty extractions
        if len(frames) == 0:
            bad_videos.append(video_path)
            continue

        # Trim start / end portions
        if len(frames) > 4:
            total = len(frames)
            start = int(total * TRIM_START_PORTION)
            end = int(total * TRIM_END_PORTION)
            frames = frames[start:end]

        if len(frames) == 0:
            bad_videos.append(video_path)
            continue

        # Choose evenly spaced frames
        indices = np.linspace(0, len(frames) - 1, MAX_FRAMES_PER_VIDEO).astype(int)
        frames = [frames[i] for i in indices]

        video_name = os.path.splitext(video)[0]
        out_video_dir = os.path.join(out_class_dir, video_name)
        os.makedirs(out_video_dir, exist_ok=True)

        for i, frame in enumerate(frames):
            cv2.imwrite(os.path.join(out_video_dir, f"{video_name}_frame{i:04d}.jpg"), frame)

print("Frame extraction complete.")
print("\nThe following videos were corrupted or unreadable:")
for b in bad_videos:
    print(" -", b)

Processing SL Classes:   6%|██▏                                    | 34/590 [00:35<08:37,  1.07it/s]

In [4]:
df_jester_train = pd.read_csv('/kaggle/input/20bn-jester/Train.csv')
df_jester_train.head()

,video_id,label,frames,label_id,shape,format
0,1,Doing other things,37,0,"(100, 176)",JPEG
1,3,Pushing Two Fingers Away,37,6,"(100, 176)",JPEG
2,6,Drumming Fingers,37,1,"(100, 176)",JPEG
3,11,Sliding Two Fingers Down,37,10,"(100, 176)",JPEG
4,14,Pushing Hand Away,37,5,"(100, 176)",JPEG


In [5]:
df_jester_train['label'].unique()

array(['Doing other things', 'Pushing Two Fingers Away',
       'Drumming Fingers', 'Sliding Two Fingers Down',
       'Pushing Hand Away', 'Shaking Hand', 'Pulling Two Fingers In',
       'Stop Sign', 'Zooming In With Two Fingers',
       'Sliding Two Fingers Up', 'Zooming Out With Two Fingers',
       'Zooming In With Full Hand', 'No gesture', 'Swiping Right',
       'Thumb Down', 'Rolling Hand Forward', 'Pulling Hand In',
       'Zooming Out With Full Hand', 'Swiping Left',
       'Rolling Hand Backward', 'Turning Hand Counterclockwise',
       'Swiping Up', 'Turning Hand Clockwise', 'Sliding Two Fingers Left',
       'Swiping Down', 'Thumb Up', 'Sliding Two Fingers Right'],
      dtype=object)

In [6]:
classes = ['Doing other things', 'No gesture']

In [14]:
classes = ['Doing other things', 'No gesture']
df_no_class = df_jester_train[df_jester_train['label'].isin(classes)].reset_index(drop=True)
df_no_class

,video_id,label,frames,label_id,shape,format
0,1,Doing other things,37,0,"(100, 176)",JPEG
1,20,Doing other things,37,0,"(100, 176)",JPEG
2,50,No gesture,37,2,"(100, 176)",JPEG
3,70,No gesture,37,2,"(100, 176)",JPEG
4,75,Doing other things,37,0,"(100, 176)",JPEG
...,...,...,...,...,...,...
6213,148008,No gesture,37,2,"(100, 176)",JPEG
6214,148075,No gesture,37,2,"(100, 176)",JPEG
6215,148079,Doing other things,37,0,"(100, 132)",JPEG
6216,148082,Doing other things,37,0,"(100, 176)",JPEG


In [16]:
train_path = "/kaggle/input/20bn-jester/Train"

# Convert video_id to strings because folder names are strings
video_ids = df_no_class["video_id"].astype(str).tolist()

# List all existing folders under Train
train_folders = set(os.listdir(train_path))

# Find matching folders
matching_folders = [vid for vid in video_ids if vid in train_folders]
len(matching_folders)

6218

In [17]:
output_path = "/kaggle/working/jester_2_classes"
os.makedirs(output_path, exist_ok=True)

for vid in matching_folders:
    src = os.path.join(train_path, vid)
    dst = os.path.join(output_path, vid)
    shutil.copytree(src, dst)

In [20]:
shutil.make_archive(output_path, 'zip', output_path)

'/kaggle/working/jester_2_classes.zip'